## Hackathon 01: Join 'plane-data.csv' to 'small.csv' to 'airport-codes.txt'

### 1) Remove any NULL rows from 'plane-data.csv'
### 2) Manually build Schema (Column names and Data types) for 'small.csv'
### 3) Remove any non-USA Countries from 'airport-codes.txt'
### 4) Write most efficient code for query below:
### 5) Query: Return 'UniqueCarrier', 'manufacturer', 'model', 'Country', 'IATA', 'DepDelay' columns. Group by 'UniqueCarrier' and count where 'DepDelay' > 3 hours
### 6) Which 'UniqueCarrier' and which 2 'IATA' have most Departure Delay flights > 3 hours?

In [0]:
%fs

ls /databricks-datasets/asa/small

path,name,size,modificationTime
dbfs:/databricks-datasets/asa/small/small.csv,small.csv,20411051,1459311537000


In [0]:
print(dbutils.fs.head("dbfs:/databricks-datasets/asa/small/small.csv"))

[Truncated to first 65536 bytes]
Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2002,12,31,2,1415,1420,2058,2117,UA,48,N598UA,283,297,272,-19,-5,LIH,SFO,2447,4,7,0,NA,0,NA,NA,NA,NA,NA
2002,12,13,5,2237,2230,711,654,UA,48,N546UA,334,324,313,17,7,SFO,BOS,2704,6,15,0,NA,0,NA,NA,NA,NA,NA
2002,12,14,6,2227,2230,638,654,UA,48,N551UA,311,324,296,-16,-3,SFO,BOS,2704,3,12,0,NA,0,NA,NA,NA,NA,NA
2002,12,15,7,2226,2230,645,654,UA,48,N544UA,319,324,295,-9,-4,SFO,BOS,2704,4,20,0,NA,0,NA,NA,NA,NA,NA
2002,12,16,1,2224,2230,641,654,UA,48,N597UA,317,324,293,-13,-6,SFO,BOS,2704,5,19,0,NA,0,NA,NA,NA,NA,NA
2002,12,17,2,2226,2230,703,654,UA,48,N598UA,337,324,294,9,-4,SFO,BOS,2704,3,40,0,NA,0,NA,NA,NA,NA,NA
2002,12,18,3,2233,2230,652,654,UA,48,N544UA,319,324,2

In [0]:
#dbutils.fs.head("dbfs:/mnt/training/asa/flights/small.csv")
print(dbutils.fs.head("/databricks-datasets/asa/planes/plane-data.csv"))

[Truncated to first 65536 bytes]
tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
N050AA
N051AA
N052AA
N054AA
N055AA
N056AA
N057AA
N058AA
N059AA
N060AA
N061AA
N062AA
N063AA
N064AA
N065AA
N066AA
N067AA
N068AA
N069AA
N070AA
N071AA
N072AA
N073AA
N074AA
N075AA
N076AA
N077AA
N078AA
N079AA
N080AA
N081AA
N082AA
N083AA
N084AA
N10156,Corporation,EMBRAER,02/13/2004,EMB-145XR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2004
N102UW,Corporation,AIRBUS INDUSTRIE,05/26/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
N10323,Corporation,BOEING,07/01/1997,737-3TO,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1986
N103US,Corporation,AIRBUS INDUSTRIE,06/18/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N104UA,Corporation,BOEING,01/26/1998,747-422,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
N104UW,Corporation,AIRBUS INDUSTRIE,07/02/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N10575,Corporation,EMBRAER,06/24/2003,EMB-145LR,Valid,Fixed Wing Multi-E

In [0]:
display(dbutils.fs.ls("/databricks-datasets/flights"))

path,name,size,modificationTime
dbfs:/databricks-datasets/flights/README.md,README.md,412,1457766852000
dbfs:/databricks-datasets/flights/airport-codes-na.txt,airport-codes-na.txt,11411,1457749605000
dbfs:/databricks-datasets/flights/departuredelays.csv,departuredelays.csv,33396236,1457749605000


In [0]:
print(dbutils.fs.head("/databricks-datasets/flights/airport-codes-na.txt"))

Yuma	AZ	USA	YUM


In [0]:
#question1
plane_data= spark.read\
.option("header",True)\
.option("inferSchema",True)\
.csv("/databricks-datasets/asa/planes/plane-data.csv")
display(plane_data)

tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
N050AA,null,null,null,null,null,null,null,null
N051AA,null,null,null,null,null,null,null,null
N052AA,null,null,null,null,null,null,null,null
N054AA,null,null,null,null,null,null,null,null
N055AA,null,null,null,null,null,null,null,null
N056AA,null,null,null,null,null,null,null,null
N057AA,null,null,null,null,null,null,null,null
N058AA,null,null,null,null,null,null,null,null
N059AA,null,null,null,null,null,null,null,null
N060AA,null,null,null,null,null,null,null,null


In [0]:
#Suppression des champs null
plane_data_clean=plane_data.dropna()
display(plane_data_clean)

tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
N10156,Corporation,EMBRAER,02/13/2004,EMB-145XR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2004
N102UW,Corporation,AIRBUS INDUSTRIE,05/26/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
N10323,Corporation,BOEING,07/01/1997,737-3TO,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1986
N103US,Corporation,AIRBUS INDUSTRIE,06/18/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N104UA,Corporation,BOEING,01/26/1998,747-422,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
N104UW,Corporation,AIRBUS INDUSTRIE,07/02/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N10575,Corporation,EMBRAER,06/24/2003,EMB-145LR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2002
N105UA,Corporation,BOEING,10/15/2007,747-451,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1994
N105UW,Corporation,AIRBUS INDUSTRIE,07/22/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N106US,Corporation,AIRBUS INDUSTRIE,08/05/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999


In [0]:
#question2
DDL_Schema = ("Year integer,Month integer,DayofMonth integer,DayOfWeek integer,DepTime string,CRSDepTime integer,ArrTime string,CRSArrTime integer,UniqueCarrier string,FlightNum integer,TailNum string,ActualElapsedTime string,CRSElapsedTime integer,AirTime string,ArrDelay string,DepDelay integer,Origin string,Dest string,Distance integer,TaxiIn integer,TaxiOut integer,Cancelled integer,CancellationCode string,Diverted integer,CarrierDelay string,WeatherDelay string,NASDelay string,SecurityDelay string,LateAircraftDelay string")

In [0]:
#exportation avecDDL_Schema
Df_flights= spark.read\
.option("header",True)\
.schema(DDL_Schema)\
.csv("dbfs:/databricks-datasets/asa/small/small.csv")
display(Df_flights)


Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2002,12,31,2,1415,1420,2058,2117,UA,48,N598UA,283,297,272,-19,-5,LIH,SFO,2447,4,7,0,NA,0,NA,NA,NA,NA,NA
2002,12,13,5,2237,2230,711,654,UA,48,N546UA,334,324,313,17,7,SFO,BOS,2704,6,15,0,NA,0,NA,NA,NA,NA,NA
2002,12,14,6,2227,2230,638,654,UA,48,N551UA,311,324,296,-16,-3,SFO,BOS,2704,3,12,0,NA,0,NA,NA,NA,NA,NA
2002,12,15,7,2226,2230,645,654,UA,48,N544UA,319,324,295,-9,-4,SFO,BOS,2704,4,20,0,NA,0,NA,NA,NA,NA,NA
2002,12,16,1,2224,2230,641,654,UA,48,N597UA,317,324,293,-13,-6,SFO,BOS,2704,5,19,0,NA,0,NA,NA,NA,NA,NA
2002,12,17,2,2226,2230,703,654,UA,48,N598UA,337,324,294,9,-4,SFO,BOS,2704,3,40,0,NA,0,NA,NA,NA,NA,NA
2002,12,18,3,2233,2230,652,654,UA,48,N544UA,319,324,286,-2,3,SFO,BOS,2704,6,27,0,NA,0,NA,NA,NA,NA,NA
2002,12,19,4,2254,2230,659,654,UA,48,N551UA,305,324,288,5,24,SFO,BOS,2704,6,11,0,NA,0,NA,NA,NA,NA,NA
2002,12,20,5,2302,2230,718,654,UA,48,N550UA,316,324,285,24,32,SFO,BOS,2704,16,15,0,NA,0,NA,NA,NA,NA,NA
2002,12,21,6,2315,2230,718,654,UA,48,N547UA,303,324,287,24,45,SFO,BOS,2704,5,11,0,NA,0,NA,NA,NA,NA,NA


In [0]:
#question3
airportsDF= spark.read\
.option("header",True)\
.option("inferSchema",True)\
.option("sep","\t")\
.csv("/databricks-datasets/flights/airport-codes-na.txt")\
.filter("Country == 'USA' ")

display(airportsDF)

City,State,Country,IATA
Aberdeen,SD,USA,ABR
Abilene,TX,USA,ABI
Akron,OH,USA,CAK
Alamosa,CO,USA,ALS
Albany,GA,USA,ABY
Albany,NY,USA,ALB
Albuquerque,NM,USA,ABQ
Alexandria,LA,USA,AEX
Allentown,PA,USA,ABE
Alliance,NE,USA,AIA


In [0]:
plane_data_clean.createOrReplaceTempView("planes_view")

In [0]:
%sql
SELECT * from planes_view

tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
N10156,Corporation,EMBRAER,02/13/2004,EMB-145XR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2004
N102UW,Corporation,AIRBUS INDUSTRIE,05/26/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
N10323,Corporation,BOEING,07/01/1997,737-3TO,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1986
N103US,Corporation,AIRBUS INDUSTRIE,06/18/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N104UA,Corporation,BOEING,01/26/1998,747-422,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
N104UW,Corporation,AIRBUS INDUSTRIE,07/02/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N10575,Corporation,EMBRAER,06/24/2003,EMB-145LR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2002
N105UA,Corporation,BOEING,10/15/2007,747-451,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1994
N105UW,Corporation,AIRBUS INDUSTRIE,07/22/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
N106US,Corporation,AIRBUS INDUSTRIE,08/05/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999


In [0]:
Df_flights.createOrReplaceTempView("flights_view")

In [0]:
%sql
SELECT * from flights_view

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2002,12,31,2,1415,1420,2058,2117,UA,48,N598UA,283,297,272,-19,-5,LIH,SFO,2447,4,7,0,NA,0,NA,NA,NA,NA,NA
2002,12,13,5,2237,2230,711,654,UA,48,N546UA,334,324,313,17,7,SFO,BOS,2704,6,15,0,NA,0,NA,NA,NA,NA,NA
2002,12,14,6,2227,2230,638,654,UA,48,N551UA,311,324,296,-16,-3,SFO,BOS,2704,3,12,0,NA,0,NA,NA,NA,NA,NA
2002,12,15,7,2226,2230,645,654,UA,48,N544UA,319,324,295,-9,-4,SFO,BOS,2704,4,20,0,NA,0,NA,NA,NA,NA,NA
2002,12,16,1,2224,2230,641,654,UA,48,N597UA,317,324,293,-13,-6,SFO,BOS,2704,5,19,0,NA,0,NA,NA,NA,NA,NA
2002,12,17,2,2226,2230,703,654,UA,48,N598UA,337,324,294,9,-4,SFO,BOS,2704,3,40,0,NA,0,NA,NA,NA,NA,NA
2002,12,18,3,2233,2230,652,654,UA,48,N544UA,319,324,286,-2,3,SFO,BOS,2704,6,27,0,NA,0,NA,NA,NA,NA,NA
2002,12,19,4,2254,2230,659,654,UA,48,N551UA,305,324,288,5,24,SFO,BOS,2704,6,11,0,NA,0,NA,NA,NA,NA,NA
2002,12,20,5,2302,2230,718,654,UA,48,N550UA,316,324,285,24,32,SFO,BOS,2704,16,15,0,NA,0,NA,NA,NA,NA,NA
2002,12,21,6,2315,2230,718,654,UA,48,N547UA,303,324,287,24,45,SFO,BOS,2704,5,11,0,NA,0,NA,NA,NA,NA,NA


In [0]:
airportsDF.createOrReplaceTempView("airports_view")

In [0]:
%sql
SELECT * from airports_view

City,State,Country,IATA
Aberdeen,SD,USA,ABR
Abilene,TX,USA,ABI
Akron,OH,USA,CAK
Alamosa,CO,USA,ALS
Albany,GA,USA,ABY
Albany,NY,USA,ALB
Albuquerque,NM,USA,ABQ
Alexandria,LA,USA,AEX
Allentown,PA,USA,ABE
Alliance,NE,USA,AIA


In [0]:
%sql
SELECT f.UniqueCarrier, p.manufacturer, p.engine_type, a.Country, a.IATA,
count(f.DepDelay) as CtDepDelay
FROM flights_view f JOIN  planes_view P
ON f.tailnum = p.tailnum
JOIN airports_view a
ON f.Origin = a.IATA
WHERE f.DepDelay>180
GROUP BY f.UniqueCarrier, p.manufacturer, p.engine_type, a.Country, a.IATA
ORDER BY CtDepDelay DESC

UniqueCarrier,manufacturer,engine_type,Country,IATA,CtDepDelay
UA,BOEING,Turbo-Fan,USA,ORD,46
UA,AIRBUS INDUSTRIE,Turbo-Jet,USA,ORD,44
DL,BOEING,Turbo-Jet,USA,ATL,20
UA,BOEING,Turbo-Jet,USA,ORD,17
DL,MCDONNELL DOUGLAS AIRCRAFT CO,Turbo-Fan,USA,ATL,14
AA,MCDONNELL DOUGLAS,Turbo-Fan,USA,ORD,10
UA,BOEING,Turbo-Fan,USA,SFO,9
AA,MCDONNELL DOUGLAS,Turbo-Fan,USA,DFW,9
UA,BOEING,Turbo-Fan,USA,DEN,8
UA,AIRBUS INDUSTRIE,Turbo-Jet,USA,DEN,6


#  Deadline for email submission is Tuesday
## To save this Notebook as file, do the following:
  
#####1) In gray vertical bar on left, click on 'Workspace' icon
#####2) Find notebook, and click on 'down' arrow on right side of it
#####3) Select 'Export' > 'IPython Notebook'
#####4) Click 'Save'
#####5) Email file to following 2 email addresses: ottmk@ucmail.uc.edu;  tharanla@mail.uc.edu;
#####   along with names of any team members so I can assign grades
#####6) You will be graded on 2 things.  I can run all the cells without Error and the Answer sets in Cells are correct.